<a href="https://colab.research.google.com/github/Natthamon-Piy/Chipotle-Visualization/blob/main/Chipotle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data import

In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns

df = pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv', sep='\t')
df.head()


,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


#Top 15 Most Ordered Item

In [3]:
item_quantities = df.groupby('item_name')['quantity'].sum()
top_15_items = item_quantities.sort_values(ascending=False).head(15)

fig = px.bar(top_15_items, x='quantity', y=top_15_items.index, orientation='h', title='Top 15 Quantity Ordered per Item')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

#Distribution of Item Prices for Top 5 Most Ordered Items

In [4]:
# Get the top 5 most ordered items
item_quantities = df.groupby('item_name')['quantity'].sum()
top_5_items = item_quantities.sort_values(ascending=False).head(5).index

# Filter the dataframe to include only the top 5 items
df_top5 = df[df['item_name'].isin(top_5_items)].copy() # Use .copy() to avoid SettingWithCopyWarning

# Convert item_price to numeric
df_top5.loc[:, 'item_price'] = df_top5['item_price'].str.replace('$', '').astype(float)

# Divide item_price by quantity if quantity is greater than 1
df_top5.loc[df_top5['quantity'] > 1, 'item_price'] = df_top5['item_price'] / df_top5['quantity']


# Sort df_top5 by the total quantity of each item for plotting order
# Create a temporary column for sorting
item_total_quantities = df_top5.groupby('item_name')['quantity'].transform('sum')
df_top5_sorted = df_top5.assign(total_quantity=item_total_quantities).sort_values('total_quantity', ascending=False)

# Create a box plot of item prices for the top 5 items with quantity in hover data
fig = px.box(df_top5_sorted, x='item_name', y='item_price',
             title='Distribution of Item Prices per Item for Top 5 Most Ordered Items (Ordered by Quantity)',
             hover_data=['quantity'],
             category_orders={"item_name": top_5_items.tolist()}) # Explicitly set category order

# fig.update_layout(yaxis={'categoryorder':'array', 'categoryarray': top_5_items[::-1]}) # Removed previous ordering attempt
fig.show()

# Salsa Type of Best selling Menu

###data prep
- extract salsa type
- extract spice level of salsa type

In [7]:
# Ensure 'choice_description_split' column exists by splitting 'choice_description'
# This code is included for self-containment if the split column is not already created
if 'choice_description_split' not in df.columns:
    df['choice_description_split'] = df['choice_description'].fillna('').str.replace(r'[\[\]]', '', regex=True).str.split(',')

# Function to extract salsa type without spice level - ADDED
def extract_salsa_type_without_spice(choices_list):
    salsa_type = None
    for choice in choices_list:
        if 'Salsa' in choice:
            # Remove spice level indicators
            salsa_type = choice.strip().replace('(Mild)', '').replace('(Hot)', '').replace('(Medium)', '').strip()
            break # Assuming only one salsa type per item for this analysis
    return salsa_type

# Create the new 'SalsaType' column if it doesn't exist - ADDED
if 'SalsaType' not in df.columns:
    df['SalsaType'] = df['choice_description_split'].apply(extract_salsa_type_without_spice)

# Replace hyphens with spaces in specific salsa names in the 'SalsaType' column - ADDED
df['SalsaType'] = df['SalsaType'].str.replace('Tomatillo-Red Chili Salsa', 'Tomatillo Red Chili Salsa').str.replace('Tomatillo-Green Chili Salsa', 'Tomatillo Green Chili Salsa')


# Function to extract spice level
def extract_spice_level(choices_list):
    spice_level = None
    for choice in choices_list:
        if 'Salsa' in choice:
            if '(Mild)' in choice:
                spice_level = 'Mild'
                break
            elif '(Hot)' in choice:
                spice_level = 'Hot'
                break
            elif '(Medium)' in choice:
                spice_level = 'Medium'
                break
    return spice_level

# Create the 'SpiceLevel' column if it doesn't exist
if 'SpiceLevel' not in df.columns:
    df['SpiceLevel'] = df['choice_description_split'].apply(extract_spice_level)


# Prepare data for sunburst chart, handling potential missing values
df_sunburst = df.copy()
df_sunburst['SalsaType'] = df_sunburst['SalsaType'].fillna('None')
df_sunburst['SpiceLevel'] = df_sunburst['SpiceLevel'].fillna('None')

# Filter to include only items that contain a salsa type
df_sunburst_filtered = df_sunburst[df_sunburst['SalsaType'] != 'None'].copy()

# Get the total quantity for each item in the filtered data
item_quantities_filtered = df_sunburst_filtered.groupby('item_name')['quantity'].sum()

# Get the top 5 most ordered items from the filtered data
top_5_items_filtered = item_quantities_filtered.sort_values(ascending=False).head(5).index

# Filter the sunburst data to include only the top 5 items
df_sunburst_top5 = df_sunburst_filtered[df_sunburst_filtered['item_name'].isin(top_5_items_filtered)].copy()

In [8]:
# Function to extract salsa type without spice level
def extract_salsa_type_without_spice(choices_list):
    salsa_type = None
    for choice in choices_list:
        if 'Salsa' in choice:
            # Remove spice level indicators
            salsa_type = choice.strip().replace('(Mild)', '').replace('(Hot)', '').replace('(Medium)', '').strip()
            break # Assuming only one salsa type per item for this analysis
    return salsa_type

# Create the new 'SalsaType' column
if 'SalsaType' not in df.columns:
    df['SalsaType'] = df['choice_description_split'].apply(extract_salsa_type_without_spice)

# Replace hyphens with spaces in specific salsa names in the 'SalsaType' column
df['SalsaType'] = df['SalsaType'].str.replace('Tomatillo-Red Chili Salsa', 'Tomatillo Red Chili Salsa').str.replace('Tomatillo-Green Chili Salsa', 'Tomatillo Green Chili Salsa')

# Filter data to include only items that contain a salsa type
df_with_salsa = df[df['SalsaType'].notna() & (df['SalsaType'] != 'None')].copy()

# Get the total quantity for each item in the filtered data
item_quantities_with_salsa = df_with_salsa.groupby('item_name')['quantity'].sum()

# Get the top 5 most ordered items from the filtered data
top_5_items_with_salsa = item_quantities_with_salsa.sort_values(ascending=False).head(5).index

# Filter the data for the top 5 items that contain salsa
df_top5_with_salsa = df_with_salsa[df_with_salsa['item_name'].isin(top_5_items_with_salsa)].copy()


# Calculate percentage of each SalsaType for each top 5 item
salsa_percentages_by_item_top5 = {}
for item in top_5_items_with_salsa:
    item_df = df_top5_with_salsa[df_top5_with_salsa['item_name'] == item]
    salsa_counts = item_df['SalsaType'].value_counts()
    if salsa_counts.sum() > 0:
        salsa_percentages = salsa_counts / salsa_counts.sum() * 100
        salsa_percentages_by_item_top5[item] = salsa_percentages.sort_values(ascending=False) # Sort by percentage descending
    else:
        salsa_percentages_by_item_top5[item] = pd.Series(dtype=float)

##Bar Chart of Salsa Type for Top 5 Most Ordered by Percentage

In [14]:
# Prepare data for plotting
plot_data_top5 = []
for item, percentages in salsa_percentages_by_item_top5.items():
    for salsa_type, percentage in percentages.items():
        if salsa_type: # Exclude empty salsa types
            plot_data_top5.append({'item_name': item, 'salsa_type': salsa_type, 'percentage': percentage})

plot_df_top5 = pd.DataFrame(plot_data_top5)

# Create a stacked bar chart with percentages shown for top 5 items with salsa - THIS BLOCK MUST COME AFTER DATA PREP
fig = px.bar(plot_df_top5, x='percentage', y='item_name', color='salsa_type',
             title='Bar Chart of Salsa Type for Top 5 Most Ordered by Percentage',
             orientation='h',
             text='percentage') # Add text labels

fig.update_layout(yaxis={'categoryorder':'array', 'categoryarray': top_5_items_with_salsa[::-1]}) # Maintain the order of items
fig.update_traces(texttemplate='%{text:.1f}%', textposition='auto')
fig.show()

##Sunburst of Salsa Type for Top 5 Most Ordered


In [15]:
# Create the sunburst chart with filtered data
fig = px.sunburst(df_sunburst_top5, path=['item_name', 'SalsaType', 'SpiceLevel'], values='quantity',
                  title='Sunburst Chart of Top 5 Most Ordered, Salsa Type, and Spice Level')
fig.show()

#สัดส่วนจำนวนquantity ที่ซื้อในแต่ละorder

In [11]:
order_quantities = df.groupby('order_id')['quantity'].sum().reset_index()
# Count the number of orders for each total quantity
quantity_counts = order_quantities.groupby('quantity').size().reset_index(name='num_orders')
# Calculate the percentage of each quantity
quantity_counts['percentage'] = (quantity_counts['num_orders'] / quantity_counts['num_orders'].sum()) * 100
# Filter out quantities less than 1%
filtered_quantity_counts = quantity_counts[quantity_counts['percentage'] >= 1]
fig = px.pie(filtered_quantity_counts, values='num_orders', names='quantity', title='Proportion of Orders by Total Quantity (Excluding less than 1%)')
fig.show()

#ช่วงราคาที่ลูกค้ามักใช้จ่ายต่อออเดอร์ (Spending per order) อยู่ในช่วงใดมากที่สุด?

In [13]:
import pandas as pd

# Convert item_price to numeric and calculate total cost per order
df['item_price'] = df['item_price'].str.replace('$', '').astype(float)
order_costs = df.groupby('order_id')['item_price'].sum().reset_index()

# Create a new column to categorize order costs with adjusted bins and ordered labels
price_bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 50, 100, float('inf')]
price_labels = ['0-5', '5-10', '10-15', '15-20', '20-25', '25-30', '30-35', '35-40', '40-50', '50-100', '100+']
order_costs['price_category'] = pd.cut(order_costs['item_price'], bins=price_bins,
                                      labels=price_labels,
                                      right=False,
                                      ordered=True) # Ensure the categories are ordered
# Explicitly set the order of categories using category_orders in Plotly
fig = px.histogram(order_costs, x='price_category', title='Distribution of Total Order Cost',
                  category_orders={'price_category': price_labels}) # Specify the desired order
fig.update_layout(xaxis_title='Total Price', yaxis_title='Number of Orders')
fig.show()